# 1

#### En el ejemplo de teoría hemos visto la forma de entrenar utilizando un trozo de datos como en la imagen "way1.png". 

(Básicamente, tener en cuenta más datos de entrenamiento [[20% de test]])

Investiga la forma de realizar el ejercicio a partir del gif "way3_sliding_window.gif":

- ¿Da mejores resultados entrenar el modelo así?

Algo muy típico en el trabajo de data scientist

![](../rnn/sliding_windows_google.png)

In [197]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

In [198]:
# Total time points
N = 3000
# Time point to partition train/test splits
Tp = int(0.8*N)   

t=np.arange(0,N)
x=(2*np.sin(0.02*t)*np.sin(0.003*t))+0.5*np.random.normal(size=N)
df = pd.DataFrame(x, columns=['Data'])
df.head()


,Data
0,0.446773
1,-0.203874
2,0.260100
3,-0.550305
4,-0.462551


In [199]:
def moving_window(data, window_length, step):
    return [data[i: i + window_length] for i in range(0, (len(data)+1)-window_length, step)]
    

data = df
y = moving_window(data, 200, 10)
y = np.asarray(y)

In [200]:
y.shape

(281, 200, 1)

In [201]:

# Time point to partition train/test splits
window_length = 200
t = int(window_length*0.8)

train, test = y[:,0:t], y[:,t:window_length]

In [202]:
print(train.shape)
test.shape

(281, 160, 1)


(281, 40, 1)

In [203]:
step = 4

# add step elements into train and test
test_list=[]
train_list = []
for i in range(len(test)):
    testing = np.append(test[i],np.repeat(test[i][-1,], step))
    test_list.append(testing)
    training = np.append(train[i],np.repeat(train[i,-1,], step))
    train_list.append(training)

test_list = np.array(test_list)
train_list = np.array(train_list)

In [204]:
test_list.shape

(281, 44)

In [210]:
# función para todas las ventanas
def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

def convertToWindowMatrix(data, step):
    X, Y = [], []
    for i in range(len(data)):
        x, y = convertToMatrix(data[i], step)
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)

In [211]:
trainX,trainY = convertToWindowMatrix(train_list,step)
testX,testY = convertToWindowMatrix(test_list,step)

In [212]:
281*160+281*4

46084

In [215]:
trainX.shape

(281, 160, 4)

In [63]:
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

def build_simple_rnn(num_units=128, embedding=10, num_dense=32, lr=0.001):    #embedding = step, lr=learning rate
    """
    Builds and compiles a simple RNN model
    Arguments:
              num_units: Number of units of a the simple RNN layer
              embedding: Embedding length
              num_dense: Number of neurons in the dense layer followed by the RNN layer
              lr: Learning rate (uses RMSprop optimizer)
    Returns:
              A compiled Keras model.
    """
    model = Sequential()
    # Long short term memory
    model.add(LSTM(units=num_units, input_shape=(1, embedding), activation="relu"))
    model.add(Dense(num_dense, activation="relu"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=lr),metrics=['mse'])
    
    return model